In [25]:
import pandas as pd
import plotly.express as px
import dash
from dash import dcc, html, Input, Output

In [26]:
# Load and preprocess data
df = pd.read_csv('/Users/mickelelia/Desktop/AMIT WORKS/Data_Science/10000 Sales Records.csv')
df['Order Date'] = pd.to_datetime(df['Order Date'])
df['Year'] = df['Order Date'].dt.year
df['Profit Margin'] = (df['Total Profit'] / df['Total Revenue']) * 100

In [27]:
# Initialize Dash app
app = dash.Dash(__name__)
app.title = "Global Sales Analytics"

In [28]:
# Define app layout
app.layout = html.Div([
    html.H1("Global Sales Performance Dashboard", style={'textAlign': 'center'}),
    
    # Filters Row
    html.Div([
        html.Div([
            html.Label("Select Region:"),
            dcc.Dropdown(
                id='region-filter',
                options=[{'label': r, 'value': r} for r in sorted(df['Region'].unique())],
                value='All',
                clearable=False
            )
        ], style={'width': '24%', 'display': 'inline-block'}),
        
        html.Div([
            html.Label("Select Country:"),
            dcc.Dropdown(id='country-filter', clearable=False)
        ], style={'width': '24%', 'display': 'inline-block'}),
        
        html.Div([
            html.Label("Select Year:"),
            dcc.Dropdown(
                id='year-filter',
                options=[{'label': y, 'value': y} for y in sorted(df['Year'].unique())],
                value=df['Year'].max(),
                clearable=False
            )
        ], style={'width': '24%', 'display': 'inline-block'}),
        
        html.Div([
            html.Label("Select Product Category:"),
            dcc.Dropdown(
                id='product-filter',
                options=[{'label': p, 'value': p} for p in sorted(df['Item Type'].unique())],
                value='All',
                clearable=False
            )
        ], style={'width': '24%', 'display': 'inline-block'})
    ], style={'margin-bottom': '20px'}),
    
    # KPI Cards
    html.Div([
        html.Div(id='total-revenue', className='kpi-card'),
        html.Div(id='total-profit', className='kpi-card'),
        html.Div(id='total-units', className='kpi-card'),
        html.Div(id='avg-margin', className='kpi-card')
    ], className='row'),
    
    # Main Charts Row
    html.Div([
        html.Div([
            dcc.Graph(id='sales-trend')
        ], className='six columns'),
        
        html.Div([
            dcc.Graph(id='product-performance')
        ], className='six columns')
    ], className='row'),
    
    # Second Charts Row
    html.Div([
        html.Div([
            dcc.Graph(id='channel-performance')
        ], className='six columns'),
        
        html.Div([
            dcc.Graph(id='country-map')
        ], className='six columns')
    ], className='row'),
    
    # Data Table
    html.Div([
        html.H3("Detailed Sales Data"),
        html.Div(id='data-table')
    ])
])

In [29]:
# CSS styling
app.css.append_css({
    'external_url': 'https://codepen.io/chriddyp/pen/bWLwgP.css'
})

In [30]:
# Callbacks for interactivity
@app.callback(
    Output('country-filter', 'options'),
    Output('country-filter', 'value'),
    Input('region-filter', 'value')
)
def update_countries(selected_region):
    if selected_region == 'All':
        countries = sorted(df['Country'].unique())
        return [{'label': c, 'value': c} for c in countries], countries[0]
    else:
        countries = sorted(df[df['Region'] == selected_region]['Country'].unique())
        return [{'label': c, 'value': c} for c in countries], countries[0]
    


In [31]:
@app.callback(
    Output('total-revenue', 'children'),
    Output('total-profit', 'children'),
    Output('total-units', 'children'),
    Output('avg-margin', 'children'),
    Input('region-filter', 'value'),
    Input('country-filter', 'value'),
    Input('year-filter', 'value'),
    Input('product-filter', 'value')
)
def update_kpis(region, country, year, product):
    filtered_df = df.copy()
    if region != 'All':
        filtered_df = filtered_df[filtered_df['Region'] == region]
    if country != 'All':
        filtered_df = filtered_df[filtered_df['Country'] == country]
    if product != 'All':
        filtered_df = filtered_df[filtered_df['Item Type'] == product]
    
    filtered_df = filtered_df[filtered_df['Year'] == year]
    
    revenue = f"${filtered_df['Total Revenue'].sum():,.0f}"
    profit = f"${filtered_df['Total Profit'].sum():,.0f}"
    units = f"{filtered_df['Units Sold'].sum():,.0f}"
    margin = f"{filtered_df['Profit Margin'].mean():.1f}%"
    
    return [
        html.Div([
            html.H3("Total Revenue"),
            html.H4(revenue)
        ]),
        html.Div([
            html.H3("Total Profit"),
            html.H4(profit)
        ]),
        html.Div([
            html.H3("Units Sold"),
            html.H4(units)
        ]),
        html.Div([
            html.H3("Avg Profit Margin"),
            html.H4(margin)
        ])
    ]

In [32]:
@app.callback(
    Output('sales-trend', 'figure'),
    Input('region-filter', 'value'),
    Input('country-filter', 'value'),
    Input('product-filter', 'value')
)
def update_trend_chart(region, country, product):
    filtered_df = df.copy()
    if region != 'All':
        filtered_df = filtered_df[filtered_df['Region'] == region]
    if country != 'All':
        filtered_df = filtered_df[filtered_df['Country'] == country]
    if product != 'All':
        filtered_df = filtered_df[filtered_df['Item Type'] == product]
    
    monthly_data = filtered_df.groupby([pd.Grouper(key='Order Date', freq='M'), 'Item Type']).agg({
        'Total Revenue': 'sum',
        'Total Profit': 'sum',
        'Units Sold': 'sum'
    }).reset_index()
    
    fig = px.line(monthly_data, x='Order Date', y='Total Revenue',
                color='Item Type', title='Monthly Sales Trend',
                hover_data=['Total Profit', 'Units Sold'])
    fig.update_layout(hovermode='x unified')
    return fig


In [33]:
@app.callback(
    Output('product-performance', 'figure'),
    Input('region-filter', 'value'),
    Input('country-filter', 'value'),
    Input('year-filter', 'value')
)
def update_product_chart(region, country, year):
    filtered_df = df[df['Year'] == year]
    if region != 'All':
        filtered_df = filtered_df[filtered_df['Region'] == region]
    if country != 'All':
        filtered_df = filtered_df[filtered_df['Country'] == country]
    
    product_data = filtered_df.groupby('Item Type').agg({
        'Total Revenue': 'sum',
        'Total Profit': 'sum',
        'Profit Margin': 'mean'
    }).reset_index().sort_values('Total Revenue', ascending=False)
    
    fig = px.bar(product_data, x='Item Type', y='Total Revenue',
                    color='Profit Margin', title='Product Performance',
                    hover_data=['Total Profit'],
                    color_continuous_scale='Bluered')
    fig.update_layout(xaxis={'categoryorder':'total descending'})
    return fig


In [34]:
@app.callback(
    Output('channel-performance', 'figure'),
    Input('region-filter', 'value'),
    Input('year-filter', 'value'),
    Input('product-filter', 'value')
)
def update_channel_chart(region, year, product):
    filtered_df = df[df['Year'] == year]
    if region != 'All':
        filtered_df = filtered_df[filtered_df['Region'] == region]
    if product != 'All':
        filtered_df = filtered_df[filtered_df['Item Type'] == product]
    
    channel_data = filtered_df.groupby('Sales Channel').agg({
        'Total Revenue': 'sum',
        'Total Profit': 'sum',
        'Units Sold': 'sum'
    }).reset_index()
    
    fig = px.pie(channel_data, values='Total Revenue', names='Sales Channel',
                    title='Sales Channel Distribution',
                    hover_data=['Total Profit'])
    fig.update_traces(textposition='inside', textinfo='percent+label')
    return fig


In [35]:
@app.callback(
    Output('country-map', 'figure'),
    Input('year-filter', 'value'),
    Input('product-filter', 'value')
)
def update_map(year, product):
    filtered_df = df[df['Year'] == year]
    if product != 'All':
        filtered_df = filtered_df[filtered_df['Item Type'] == product]
    
    map_data = filtered_df.groupby(['Country', 'Region']).agg({
        'Total Revenue': 'sum',
        'Total Profit': 'sum'
    }).reset_index()
    
    fig = px.choropleth(map_data, locations='Country', locationmode='country names',
                        color='Total Profit', hover_name='Country',
                        hover_data=['Region', 'Total Revenue'],
                        title='Profit by Country',
                        color_continuous_scale='Viridis')
    fig.update_layout(geo=dict(showframe=False, showcoastlines=False))
    return fig


In [36]:
@app.callback(
    Output('data-table', 'children'),
    Input('region-filter', 'value'),
    Input('country-filter', 'value'),
    Input('year-filter', 'value'),
    Input('product-filter', 'value')
)
def update_table(region, country, year, product):
    filtered_df = df.copy()
    if region != 'All':
        filtered_df = filtered_df[filtered_df['Region'] == region]
    if country != 'All':
        filtered_df = filtered_df[filtered_df['Country'] == country]
    if product != 'All':
        filtered_df = filtered_df[filtered_df['Item Type'] == product]
    
    filtered_df = filtered_df[filtered_df['Year'] == year]
    
    return dash.dash_table.DataTable(
        data=filtered_df.sort_values('Order Date', ascending=False).head(20).to_dict('records'),
        columns=[{'name': i, 'id': i} for i in filtered_df.columns],
        page_size=10,
        style_table={'overflowX': 'auto'},
        style_cell={
            'height': 'auto',
            'minWidth': '100px', 'width': '120px', 'maxWidth': '180px',
            'whiteSpace': 'normal'
        }
    )


In [37]:
app.run_server()

/Users/mickelelia/Desktop/AMIT WORKS/venv/lib/python3.13/site-packages/dash/resources.py:61: UserWarning:

You have set your config to `serve_locally=True` but A local version of https://codepen.io/chriddyp/pen/bWLwgP.css is not available.
If you added this file with `app.scripts.append_script` or `app.css.append_css`, use `external_scripts` or `external_stylesheets` instead.
See https://dash.plotly.com/external-resources



[2025-03-31 21:25:08,950] ERROR in app: Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/Users/mickelelia/Desktop/AMIT WORKS/venv/lib/python3.13/site-packages/flask/app.py", line 1473, in wsgi_app
    response = self.full_dispatch_request()
  File "/Users/mickelelia/Desktop/AMIT WORKS/venv/lib/python3.13/site-packages/flask/app.py", line 882, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/Users/mickelelia/Desktop/AMIT WORKS/venv/lib/python3.13/site-packages/flask/app.py", line 880, in full_dispatch_request
    rv = self.dispatch_request()
  File "/Users/mickelelia/Desktop/AMIT WORKS/venv/lib/python3.13/site-packages/flask/app.py", line 865, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)  # type: ignore[no-any-return]
           ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~^^^^^^^^^^^^^
  File "/Users/mickelelia/Desktop/AMIT WORKS/venv/lib/python3.13/site-packages/da